In [1]:
from pathlib import Path
import libs.analysis
import libs.ensemble
import libs.vars
import numpy as np
import xarray

import warnings
warnings.filterwarnings('ignore')

experiment = 'ssp585'
#variables = [v for v in libs.vars.variables() if v['variable_id'] == 'pr']
variables = libs.vars.variables()

/home/users/woodward/.conda/envs/climate/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
def generate_ensemble_time_series(
    ensemble,
    filename_prefix,
    weight,
    weighting_method,
    weighting_process,
    attrs
):
    ensemble_time_series, ensemble_time_series_smooth = libs.ensemble.time_series_weighted(
        ensemble,
        weight,
        weighting_method=weighting_method,
        weighting_process=weighting_process
    )

    all_series = [
        { 'series': ensemble_time_series, 'suffix': '' },
        {
            'series': ensemble_time_series,
            'process': lambda x: libs.analysis.climatology_monthly(x, '1980-01-01', '2011-01-01'), 
            'suffix': '_delta_1980-2010'
        },
        { 'series': ensemble_time_series_smooth, 'suffix': '_smooth' },
    ]

    for series_item in all_series:
        series = series_item['series']
        process = series_item['process'] if 'process' in series_item else lambda x: x

        data_vars = {}
        ensemble = None
        for i, item in enumerate(series):
            processed_data = process(item['data'])
            processed_data.attrs['color'] = item['color']
            ensemble = ensemble + processed_data if i > 0 else processed_data
            processed_data = processed_data.drop_vars('height', errors='ignore')
            data_vars[item['label']] = processed_data

        ensemble /= len(series)
        ensemble.attrs['color'] = '#000'
        ensemble.attrs['label'] = 'Ensemble mean'
        data_vars['Ensemble mean'] = ensemble

        ds = xarray.Dataset(
            data_vars=data_vars, 
            coords=series[0]['data'].drop_vars('height', errors='ignore').coords,
            attrs=attrs
        )

        d_start = ds.time.values[0].strftime('%Y%m')
        d_end = ds.time.values[-1].strftime('%Y%m')
        suffix = series_item['suffix']
        filename = f'{filename_prefix}_{d_start}-{d_end}{suffix}.nc'
        filepath = Path(f'_data/_cache/{filename}')
        if filepath.exists():
            #print('-> Already, exists. Skipping')
            continue

        print('Saving to:', filepath)
        filepath.parent.mkdir(parents=True, exist_ok=True)
        ds.to_netcdf(
            filepath,
            engine='netcdf4',
            unlimited_dims=['time']
        )


def get_ensemble_and_generate_time_series(
    component,
    experiment,
    text,
    units,
    variable_id,
    weighting_method,
    weighting_process,
    preprocess=None
):
    kwargs = {
        'component': component,
        'experiment': experiment, 
        'variable_id': variable_id
    }
    
    if preprocess != None:
        kwargs['preprocess'] = preprocess

    ensemble, weight = libs.ensemble.get_and_preprocess(**kwargs)
    
    regions = libs.vars.nsidc_regions()
    path_nsidc_mask = '_data/_cache/NSIDC_Regions_Masks_Ocean_nearest_s2d.nc'
    nsidc_mask = xarray.open_mfdataset(paths=path_nsidc_mask, combine='by_coords').mask
    
    for i, region in enumerate(regions):
        region_name = region['label']
        region_slug = region_name.replace(' ', '_')
        ensemble_masked = [{ 
            'color': item['color'],
            'data': item['data'].where(np.isin(nsidc_mask.values, region['values'])),
            'label': item['label']
        } for item in ensemble]
        
        generate_ensemble_time_series(
            ensemble_masked,
            filename_prefix=f'{variable_id}/{variable_id}_{experiment}_{region_slug}',
            weight=weight,
            weighting_method=weighting_method,
            weighting_process=weighting_process,
            attrs={
                'description': f'Monthly {text} in {region_name}',
                'units': units,
                'region': region_name,
                'variable_id': variable_id
            }
        )

In [3]:
[get_ensemble_and_generate_time_series(experiment=experiment, **v) for v in variables];

Saving to: _data/_cache/tas/tas_ssp585_All_198001-210012.nc
Saving to: _data/_cache/tas/tas_ssp585_All_198001-210012_delta_1980-2010.nc
Saving to: _data/_cache/tas/tas_ssp585_All_198001-210012_smooth.nc
Saving to: _data/_cache/tas/tas_ssp585_Arctic_Ocean_198001-210012.nc
Saving to: _data/_cache/tas/tas_ssp585_Arctic_Ocean_198001-210012_delta_1980-2010.nc
Saving to: _data/_cache/tas/tas_ssp585_Arctic_Ocean_198001-210012_smooth.nc
Saving to: _data/_cache/tas/tas_ssp585_Labrador_198001-210012.nc
Saving to: _data/_cache/tas/tas_ssp585_Labrador_198001-210012_delta_1980-2010.nc
Saving to: _data/_cache/tas/tas_ssp585_Labrador_198001-210012_smooth.nc
Saving to: _data/_cache/tas/tas_ssp585_Greenland_198001-210012.nc
Saving to: _data/_cache/tas/tas_ssp585_Greenland_198001-210012_delta_1980-2010.nc
Saving to: _data/_cache/tas/tas_ssp585_Greenland_198001-210012_smooth.nc
Saving to: _data/_cache/tas/tas_ssp585_Barents_198001-210012.nc
Saving to: _data/_cache/tas/tas_ssp585_Barents_198001-210012_del